# Drawing Toronto Neigborhoods

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Here we import postal codes of Toronto

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content, "lxml")

The table which we need to extract is within the node `<table class="wikitable sortable">`, so we find it

In [3]:
table = soup.find("table", class_="wikitable sortable")

In [4]:
table_rows = table.find_all('tr') #parsing rows

Parsing and cleaning data

In [5]:
data=[]
for tr in table_rows[1:]: #in table_rows[0] we have column titles
    row=[]
    tds = tr.find_all('td')
    for td in tds:
        row.append(td.text.rstrip()) #rstrip for removing \n
    if(row[1]!='Not assigned'): #cleaning Not assigned boroughs
        if(row[2]=='Not assigned'):
            row[2]=row[1]
        data.append(row)

In [6]:
data[:5]

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights']]

Cleaning duplicate postal codes and generating dataframe

In [7]:
cleaned_data = []
for idx, value in enumerate(data[:-1]):
    if (data[idx][0] == data[idx+1][0]):
        data[idx+1][2]= data[idx][2] + ", " + data[idx + 1][2]
    else:
        cleaned_data.append(data[idx])
cleaned_data.append(data[-1])

In [8]:
df = pd.DataFrame(cleaned_data, columns = ["Postcode", "Borough", "Neighborhood"])

In [9]:
df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


Adding geo coordinates to dataframe

In [10]:
geo = pd.read_csv("Geospatial_Coordinates.csv")
geo = geo.rename(columns={'Postal Code': 'Postcode'})
df = pd.merge(df, geo, how='left', on=['Postcode'], )

Generating map with neighborhoods

In [11]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronro are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronro are 43.653963, -79.387207.


In [12]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto


In [13]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto